(sec:md)=
# Molecular dynamics

A fully quantum mechanical treatment is very costly, and thus only applicable for relatively small systems. This means that unless new approximations are introduced, many systems that are of interest will be far beyond the capabilities of today’s computers. A small piece of protein from the human body can consist of several thousand atoms, and even very small systems can increase by hundreds of atoms when a solvent is added.

The simulation of large proteins and long time-scales can instead be consider using molecular dynamics (MD), in which we use simple Newtonian physics on entities (e.g. atoms) constructed to emulate the behavior of the molecular system. In this chapter we will introduce the basics for running an MD simulation with [openMM](https://openmm.org/).

## OpenMM
OpenMM offers a high-performance toolkit for molecular simulations. It consists of a library of simulation features and an application layer, allowing end-users to use the program directly. The application layer allows openMM to be run as a stand-alone program, which is what we exploit in this course. 

### Running a basic MD simulation with openMM
The following example is taken from the [openMM user guide](http://docs.openmm.org/latest/userguide/application/02_running_sims.html#a-first-example).

Roughly, the script below carries out five actions:
1. loads a PDB file called input.pdb defining a biomolecular system
2. parameterizes the system using the Amber14 force field and TIP3P-FB water model
3. minimizes the energy
4. runs a simulation of 10,000 steps with a Langevin integrator
5. saves a snapshot frame to a PDB file called output.pdb every 1000 time steps.

In [ ]:
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

pdb = PDBFile('input.pdb')
forcefield = ForceField('amber14-all.xml', 'amber14/tip3pfb.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,
        nonbondedCutoff=1*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picosecond, 0.004*picoseconds)
simulation = Simulation(pdb.topology, system, integrator)
simulation.context.setPositions(pdb.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('output.pdb', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(10000)